# Quick experiment: Cropping point cloud with open 3D

In [17]:
import open3d as o3d
import os
from typing import List
from tqdm import tqdm
import numpy as np
import laspy

In [7]:
os.chdir('/scratch/bingxin/project/ImpliCity_Regress/ImpliCityFork')
input_pc_folder = "data/source_data/ZUR1/Point_Clouds"
input_pc_paths: List = [os.path.join(input_pc_folder, _path) for _path in os.listdir(input_pc_folder)]

In [15]:
def load_las_as_numpy(las_path: str) -> np.ndarray:
    """
    Load .las point cloud and convert into numpy array
    This one is slow, because laspy returns a list of tuple, which can't be directly transformed into numpy array
    Args:
        las_path: full path to las file

    Returns:

    """
    with laspy.open(las_path) as f:
        _las = f.read()
    x = np.array(_las.x).reshape((-1, 1))
    y = np.array(_las.y).reshape((-1, 1))
    z = np.array(_las.z).reshape((-1, 1))
    points = np.concatenate([x, y, z], 1)
    # points = _las.points.array
    # points = np.asarray(points.tolist())[:, 0:3].astype(np.float)
    return points

In [18]:
merged_pts: np.ndarray = np.empty((0, 3))
for _full_path in tqdm(input_pc_paths, desc="Loading point clouds"):
    # _temp_points = load_pc(_full_path)
    _temp_points = load_las_as_numpy(_full_path)

    merged_pts = np.append(merged_pts, _temp_points, 0)


Loading point clouds: 100%|██████████| 26/26 [00:39<00:00,  1.52s/it]


In [44]:
o3d_device = o3d.core.Device("CPU:0")
pcd = o3d.t.geometry.PointCloud(merged_pts)

# pcd.points = o3d.utility.Vector3dVector(merged_pts)

In [65]:
pcd.point['label'] = np.random.rand(merged_pts.shape[0])

In [66]:
pcd

PointCloud on CPU:0 [165928107 points (Float64)] Attributes: label (dtype = Float64, shape = {165928107}).

In [53]:
pcd.get_max_bound()

[465450. 5.24994e+06 936.145]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x55743d7005c0]

In [57]:
p_min = pcd.get_min_bound()
p_max = p_min + np.array([64., 64., 200.])

In [69]:
out = o3d.geometry.PointCloud.crop_point_cloud(p_min, p_max)

AttributeError: type object 'open3d.cuda.pybind.geometry.PointCloud' has no attribute 'crop_point_cloud'

In [59]:
o3d.utility.Vector4iVector()

[463210. 5.24815e+06 383.607]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x55743bbde030]